# Quadratic Sieve Implementation

## Preliminaries

Imports

In [64]:
import math

### Part 1.1: Strong Pseudoprime Test

A few notes to consider when implementing the strong pseudoprime test:  
* There are no strong Carmichael Numbers - this was shown in class and homework  
* A composite number n is a strong pseudoprime to at most one quarter of all bases below n  

So, the test will be iterated over a number of bases to ensure (very high probability) primality  
For the test, there is no composite that passses the test for 2, 325, 9375, 28178, 450775, 9780504, and 1795265022  
* https://en.wikipedia.org/wiki/Strong_pseudoprime#cite_note-10  



In [40]:
# FAST MODULAR EXPONENTIATION
# b = base for exponentiation
# exp = exponent
# n = modulo
# returns result of fast modular exponentiation
def fastExp(b, t, n):
    res = 1
    for i in range(0, t):
        res = (res * b) % n
    # Check result for strong pseudoprime context
    if (res == (n - 1)):
        return -1
    return res

# STRONG PSEUDOPRIME TEST
# n = questionable prime
# bases = array of bases for test
# returns 0 on passed, -1 on failed test
def strongPseudo(n, bases):
    for i in range(len(bases)):
        t = n - 1
        while (not(t % 2)):
            res = fastExp(bases[i], t, n)
            if (res == 1):
                t = t // 2
                # Continue base checking
                continue
            elif (res == -1):
                # Assume probable pseudoprime of current base
                break
            else:
                # Number is composite - fails test 
                return -1
    return 0

n = 3004879 
bases = [2, 325, 9375, 28178]
# print(fastExp(3, 3962, 31697))
# NOTE: Write a loop to exemplify 

print(strongPseudo(n, bases))

-1


### Part 1.2: Trial Division

Notes on Trial Division:  
* Trial Division is used to factor out primes of some number, n, up to some max value.  
* The unfactored portion (after max) will be called 's'  
* The result of this module will be used in finding a root modulo p (Tonelli's Algorithm)

In [47]:
# TRIAL DIVISION
# n = Number to divide
# parr = array of primes 
# returns arr (trial prime factorization exponents), n (AKA 's' the unfactored portion)
def trialDiv(n, parr):
    arr = [0] * len(parr)
    # Check each prime in prime array input
    for i in range(len(parr)):
        # Keep dividing prime out until no longer can
        while (True):
            if (n % parr[i] == 0):
                arr[i] = arr[i] + 1
                n = n // parr[i]
            else:
                break
    return arr, n

# NOTE: Write loop to exemplify 
print(trialDiv(31696, [2]))

([4], 1981)


### Part 2.1: Inverse Modulo

Notes on Inverse Modulo:  
* Implement Extended Euclidean Algorithm for finding some inverse of a modulo n  


In [ ]:
# EXTENDED EUCLIDEAN ALGORITHM
# a = Input number
# n = modulus
# Returns Inverse mod n (inv)
def extendedEuclid(a, n):
    inv = 1
    div = 1
    m = n
    stack = []
    # First sequence: find two numbers where gcd = 1
    div = n // a
    stack.insert(0, [(m, 1), (a, div)])
    b = m % a
    m = a
    a = b
    while (True):
        div = a // b
        stack.insert(0, [(a, 1), (a, div)])
        b = m % a
        m = a
        a = b
        if (b):
            break
    # Second sequence: get inverse
    target = stack[0]
    stack.pop(0)
    for i in range(len(stack)):
        
    if (inv < 0):
        inv = n + inv
    return inv

In [147]:
def extEuclid(a,b):
    n = b
    prevx, x = 1, 0; prevy, y = 0, 1
    while b:
        q = a // b
        x, prevx = prevx - q*x, x
        y, prevy = prevy - q*y, y
        a, b = b, a % b
    if (prevx < 0):
        return n + prevx
    return prevx

print(extEuclid(150, 180))

179


### Part 2.2: Tonelli's Algorithm 

Calculating Jacobi symbols will be necessary for this module  
Notes on Jacobi Symbols:  
* j

In [102]:
# JACOBI SYMBOLS
# n = integer in question
# p = prime modulus
# Returns 1 (on residue exists), -1 (on non-residue), 0 (p divides n)
def jacobi (a, n):
    j = 1
    # Main Loop
    while a != 0:
        # Loop for pulling out factors of two
        while a % 2 == 0:
            a = a // 2
            if ((n % 8 == 3) or (n % 8 == 5)):
                j = j * -1
        # Swap a, n
        temp = a
        a = n
        n = temp
        if ((a % 4 == 3) and (n % 4 == 3)):
            j = j * -1
        a = a % n
    if n == 1:
        return j
    return 0

a = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
n = 9991
for i in a:
    print("Jacobi Symbol {} / {} = {}".format(n, i, jacobi(n, i)))

Jacobi Symbol 9991 / 2 = 1
Jacobi Symbol 9991 / 3 = 1
Jacobi Symbol 9991 / 5 = 1
Jacobi Symbol 9991 / 7 = 1
Jacobi Symbol 9991 / 11 = 1
Jacobi Symbol 9991 / 13 = -1
Jacobi Symbol 9991 / 17 = -1
Jacobi Symbol 9991 / 19 = 1
Jacobi Symbol 9991 / 23 = 1
Jacobi Symbol 9991 / 29 = -1


Notes on Tonelli's Algorithm:  
This method is used for finding a root mod some prime, which will be very useful in the Quadratic Sieve   
* The following example detatils the steps  
  * Take x^2 = 9 mod 73  
  1. Check if 9 is a quadratic residue of 73 (via Jacobi Symbols)
  2. Use Trial Division (see 1.2) to obtain some s, t pair  
      * 73 - 1 = 72, 72 = 2^3 * 9 (s = 3, t = 9)  
  3. Find a non-residue (b mod p) via repeated jacobi symbols ((b/p) = -1) (b is new base - see below)
  4. Find an (even) i which satisfies (EQ: (a/(b^i))^(2^(s-k)*t) = 1 mod p), where 'a' is the square, 'p' is the prime, and 'b' is the base of i
      * Start with i = 2
      * Check if ik satisfies the EQ
            * If it does satisfy for remainder 1, then i(k+1) = i(k)    
             * O.W. i(k+1) = i(k) + 2^k
      * The above is computed for k = 1 -> s (see part 2)
  5. Take the final i(k = s) and compute: (b^(i/2)) * ((a/(b^i))^((t+1)/2))
  6. Pray 

In [128]:
# TONELLI'S ALGORITHM 
# a = number 
# n = modulus 
# Returns a root mod n (r), o.w. -1
def tonelli(a, p):
    # Check Jacobi Symbol (a/p) = 1
    if (jacobi(a, p) != 1):
        print("{} is not a quadratic residue mod {}".format(a, p))
        return -1
    # Get s, t
    s_arr, t = trialDiv(p - 1, [2])
    s = s_arr[0]
    # Find a non-residue 
    b = 2
    while (True):
        # print("b = {}".format(b))
        if (jacobi(b, p) == -1):
            break
        b = b + 1
    # Finding ik
    i_arr = []
    i = 2
    k = 2
    while (k <= s):
        print("i = {}, k = {}".format(i, k))
        base = a / (pow(b, i))
        # Check base - Inverse if needed
        if (base < 1):
            base = math.floor(pow(base, -1))
            base = extEuclid(base, p)
        e = pow(2, (s-k)) * t
        # print("base = {}, exponent = {}".format(base, e))
        # Current i(k) worked 
        if (fastExp(base, e, p) == 1):
            i_arr.append(i)
        # Need to update i(k) to i(k) + 2^k
        else: 
            i = i + pow(2, k)
            base = a / (pow(b, i))
            if (base < 1):
                base = math.floor(pow(base, -1))
                base = extEuclid(base, p)
            if (fastExp(base, e, p) != 1):
                print("Error in confirming i{}".format(k))
                break
        k = k + 1
    # Compute final root mod p
    # Define as B * (A^T)
    #   B = b^(i/2)
    #   A = a / (b^i)
    #   T = (t + 1) / 2
    B = pow(b, i // 2)
    A = a / pow(b, i)
    if (A < 1):
        A = math.floor(pow(A, -1))
        A = extEuclid(A, p)
    T = (t + 1) // 2
    B = B % p
    print(B)
    A = fastExp(A, T, p)
    print(A)
    return (B * A)

print(tonelli(8, 193))



6 3
5
i = 2, k = 2
i = 2, k = 3
Error in confirming i3
37
65
2405


In [145]:
a = 8/pow(5, 6)
print(a)
i = math.floor(pow(a, -1))
print(i)
i = extEuclid(i+1, 193)
print(i)

0.000512
1953
185


## Main Modules for Sieving

### Part 3: Gaussian Elimination Modulo 2

Notes on Gaussian Elimination: 

### Part 4: Generate Factor Base

Before finding the factor base, need a means of getting all primes up to some integer, B  
Sieve of Eratosthenes:    
* Starting with p = 2, color all multiples of p up to b 
* For each iteration, the next non-colored integer is prime  
* Visual: https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes


In [ ]:
# Sieve of Eratosthenes
# b = Limit of base
# Returns primes, an array of primes up to B
def eratosthenes(b):
    r = int(math.ceil(math.sqrt(b)))
    p = 2
    primes = []
    nums = [0] * b
    nums[0] = 1
    num = 1
    while (True):
        loc = p
        while (True):
            loc = p * num
            if (loc > b):
                break
            nums[loc-1] = 1
            num = num + 1
        primes.append(p)
        num = 1
        # Find next prime 
        loc = p
        found = 0
        while (loc <= b):
            # Number not yet colored - next prime
            if (nums[loc-1] == 0):
                p = loc
                found = 1
                break
            loc = loc + 1
        # Sought up to root and no more primes - done
        if (found == 0):
            break  
    return primes

arr = eratosthenes(1000)
print(len(arr))
    

Notes on Factor Bases:  
* Find list of primes less than integer, B (via sieve of eratosthenes)  
* Ensure residues exist for n (number to be factored) and p (a prime element in the list)

In [100]:
# FACTOR BASES
# b = Upper prime bound
# n = number to be factored
# Returns base, a list of primes
def factorBase(b, n):
    base = []
    primes = eratosthenes(b)
    for p in primes:
        if (jacobi(n, p) == 1):
            base.append(p)
    return base

print(factorBase(30, 73))


2
3
5
7
11
13
17
19
23
29
[2, 3, 19, 23]


### Part 5: Sieve

Notes on Sieve Implementation:

## Examples of Factoring using the Quadratic Sieve

### Part 6: Wrap Up